In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

DATA_DIR = 'data'

In [ ]:
bank = pd.read_csv('data/bank_by_zip.csv')

In [ ]:
bank.dropna(inplace=True)

In [ ]:
race = 'black'
normalize_period = '2016Q2'
top_n = 300

df_demograph = pd.read_csv(os.path.join(DATA_DIR, 'cps_data/demographic_cleaned.csv'), dtype={'zip3': str})
black_zip = df_demograph.groupby('zip3')[f'is_{race}'].mean()
black_zip = black_zip.sort_values(ascending=False).index[:top_n].tolist()

In [ ]:
len(set(black_zip))

In [ ]:
loan_df = bank[['lncrcd', 'lncon', 'date', 'zip3']].copy()
loan_df['issue_q'] = pd.to_datetime(loan_df['date']).dt.to_period('Q')
loan_df['zip3'] = loan_df['zip3'].astype(str)
loan_df['lncon'] = loan_df['lncon'].rolling(4).mean()
loan_df['lncrcd'] = loan_df['lncrcd'].rolling(4).mean()
# loan_df['lncon'] = np.log(loan_df['lncon'] + 1)
# loan_df['lncrcd'] = np.log(loan_df['lncrcd'] + 1)

In [ ]:
# loan_df.fillna(loan_df.mean(), inplace=True)

In [ ]:
loan_black = loan_df[loan_df['zip3'].isin(black_zip)]
loan_non_black = loan_df[~loan_df['zip3'].isin(black_zip)]

In [ ]:
mask = (pd.to_datetime(loan_black['date']) >= '2015-01-01')
loan_black = loan_black[mask]
mask = (pd.to_datetime(loan_non_black['date']) >= '2015-01-01')
loan_non_black = loan_non_black[mask]

In [ ]:
loan_black_amt = loan_black.groupby('issue_q')['lncon'].sum()
loan_black_amt /= loan_black_amt[normalize_period]
loan_non_black_amt = loan_non_black.groupby('issue_q')['lncon'].sum()
loan_non_black_amt /= loan_non_black_amt[normalize_period]

fig = plt.figure(figsize=(8,6))
loan_black_amt.plot(label=f'{race}')
loan_non_black_amt.plot(label=f'non-{race}')
plt.axvline(x=normalize_period, color = 'black',  linestyle='--')
plt.title(f'Bank Loan Amount around LC Scandal')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Normalized Average Funded Amout')
plt.ylim(0.5, 1.5)
# plt.show()
plt.savefig('bank_2016q2_black.png')